#  Laboratory 9: Coreference

## Installs

In [ ]:
# messed up the initial instalation of neuralcoref
!pip install folium==0.2.1
!python -m spacy download en_core_web_sm
!pip install neuralcoref

In [ ]:
!pip install spacy==2.1.0 --ignore-installed

# Imports

In [ ]:
# imports
import spacy
import neuralcoref
#Loading the Languages
nlp = spacy.load('en_core_web_sm')
neuralcoref.add_to_pipe(nlp)

# CoreNLPParser

In [3]:
# Local language file needs to be downloaded and then a java service needs to be started in order to make the API call
from nltk.parse import CoreNLPParser
tagger = CoreNLPParser(url='http://localhost:9000')
tagger.parser_annotator='tokenize,ssplit,pos,lemma,ner,depparse,coref'

In [4]:
def showMyChains(CHAINS):
    for idx, chains in enumerate(CHAINS):
        print(f"Chain {idx+1}")
        for x in CHAINS[chains]:
            print(x['text'],x['type'],x['gender'],x['animacy'],x['number'],x['isRepresentativeMention'])

In [6]:
text = "Alice was beginning to get very tired of sitting by her sister on the bank,and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, ‘and what is the use of a book,’ thought Alice ‘without pictures or conversations?"
res = tagger.api_call(text)
mychains = res['corefs']
showMyChains(mychains)

Chain 1
Alice PROPER FEMALE ANIMATE SINGULAR False
her PRONOMINAL FEMALE ANIMATE SINGULAR False
she PRONOMINAL FEMALE ANIMATE SINGULAR False
her PRONOMINAL FEMALE ANIMATE SINGULAR False
Alice ` without pictures or conversations PROPER FEMALE ANIMATE SINGULAR True
Chain 2
her sister NOMINAL FEMALE ANIMATE SINGULAR True
her sister NOMINAL FEMALE ANIMATE SINGULAR False
Chain 3
the book her sister was reading NOMINAL NEUTRAL INANIMATE SINGULAR True
it PRONOMINAL NEUTRAL INANIMATE SINGULAR False
it PRONOMINAL NEUTRAL INANIMATE SINGULAR False


## HuggingFace Coreference

In [ ]:
text = nlp("Alice was beginning to get very tired of sitting by her sister on the bank,and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, ‘and what is the use of a book,’ thought Alice ‘without pictures or conversations?")

# any of the coreferences has been resolved?
text._.has_coref

# Clusters of the Corereference 
print(*text._.coref_clusters)


# the resolved sentence replacing the corerefering mention is replaced by the main
# mention in the associated cluster 
print(*nlp(text._.coref_resolved).sents)


# showing the scores for each of the coreferences 
scores = text._.coref_scores
for value in scores.values():
  print(value)

# Report

What we can see from this is the coreference chains for each of the sentences that the Core NLP Parser is able to identify the chains, the type, gender, the animacy and whether the word is a "isRepresentativeMention". This helps us to make use of how the words related to each other, as well as the pertenence of this. In this case it identifies that "Sister" has a pertenance to Alice, and that they are related. Moreover, whenever it mentions "her" it relates to Alice back again.

Additionally, what I find interesting is that it divides into it's own sentences even tho it is a run-on sentence. Since there are only two punctuation signs (:, ?), and it decided to split it into 3 distinct chains. 

In the beginning I was unsure what Coreferencing would be useful for, but now I understand that in order to translate our language into something more machine readable we need to adjust the rules that govern it. In this case, we want to show how the words relate in a given sentence and to see that we are able to identify when two or more words refer to the same thing or person. One application that I could think off would be for a Chatbot, when a "temporal" line needs to be followed in order to ask further follow-up questions i.e. Customer Service or Support. 

On the other hand, we also used the HuggingFace Coreference, here we saw that it gives a numerical value to each of the texts and creates different chains. It is passically parsing a probabilistic tree with the corresponding proababilities for a given word. The way I understood it, it would also increase the understanding of how two or more words related to one object/person/thing. 